# Topic 15: Variable Scope and Namespaces

## Overview
Understanding how Python manages variable access and scope is crucial for writing maintainable code and avoiding common pitfalls.

### What You'll Learn:
- Local, enclosing, global, and built-in scope (LEGB rule)
- Global and nonlocal keywords
- Closures and factory functions
- Namespace management
- Common scope-related pitfalls
- Best practices for variable scope

---

## 1. Understanding the LEGB Rule

Python follows the LEGB rule for scope resolution:

In [ ]:
# LEGB Rule: Local, Enclosing, Global, Built-in
print("LEGB Rule Demonstration:")
print("=" * 25)

# Built-in scope (B)
print(f"Built-in function len: {len}")
print(f"Built-in function max: {max}")

# Global scope (G)
global_var = "I'm global"
print(f"\nGlobal variable: {global_var}")

# Enclosing scope (E) and Local scope (L)
def outer_function():
    """Demonstrate enclosing and local scope"""
    enclosing_var = "I'm in enclosing scope"
    
    def inner_function():
        local_var = "I'm local"
        print(f"  Local: {local_var}")
        print(f"  Enclosing: {enclosing_var}")
        print(f"  Global: {global_var}")
        print(f"  Built-in len: {len('test')}")
    
    inner_function()
    print(f"  Can't access local_var from outer: {locals()}")

print("\nLEGB resolution in nested function:")
outer_function()

# Variable shadowing
def demonstrate_shadowing():
    """Show how inner variables can shadow outer ones"""
    global_var = "I'm shadowing global"
    len = "I'm shadowing built-in len"
    
    print(f"\nInside function:")
    print(f"  global_var: {global_var}")
    print(f"  len: {len}")
    
    # This would cause an error now:
    # print(len([1, 2, 3]))  # TypeError: 'str' object is not callable

demonstrate_shadowing()
print(f"\nOutside function (global still intact):")
print(f"  global_var: {global_var}")
print(f"  len function works: {len([1, 2, 3])}")

# Scope resolution order
test_var = "global"

def outer():
    test_var = "enclosing"
    
    def middle():
        test_var = "middle"
        
        def inner():
            test_var = "local"
            print(f"    Inner sees: {test_var}")
        
        def inner_no_local():
            print(f"    Inner (no local) sees: {test_var}")
        
        print(f"  Middle sees: {test_var}")
        inner()
        inner_no_local()
    
    print(f"Outer sees: {test_var}")
    middle()

print(f"\nScope resolution demonstration:")
print(f"Global: {test_var}")
outer()

## 2. Global and Nonlocal Keywords

Modifying variables in outer scopes:

In [ ]:
# Global and nonlocal keywords
print("Global and Nonlocal Keywords:")
print("=" * 30)

# Global keyword
counter = 0
print(f"Initial global counter: {counter}")

def increment_counter():
    """Increment global counter"""
    global counter
    counter += 1
    print(f"  Inside function: {counter}")

def try_increment_without_global():
    """This would create a local variable"""
    # counter += 1  # UnboundLocalError!
    counter = 10  # This creates a local variable
    print(f"  Local counter: {counter}")

print("\nUsing global keyword:")
increment_counter()
print(f"After increment: {counter}")

print("\nWithout global keyword:")
try_increment_without_global()
print(f"Global counter unchanged: {counter}")

# Multiple global variables
config_debug = False
config_timeout = 30

def update_config(debug=None, timeout=None):
    """Update global configuration"""
    global config_debug, config_timeout
    
    if debug is not None:
        config_debug = debug
    if timeout is not None:
        config_timeout = timeout
    
    print(f"  Config updated: debug={config_debug}, timeout={config_timeout}")

print(f"\nInitial config: debug={config_debug}, timeout={config_timeout}")
update_config(debug=True, timeout=60)
print(f"Final config: debug={config_debug}, timeout={config_timeout}")

# Nonlocal keyword
def create_counter():
    """Factory function creating a counter with closure"""
    count = 0
    
    def increment():
        nonlocal count
        count += 1
        return count
    
    def decrement():
        nonlocal count
        count -= 1
        return count
    
    def get_count():
        return count
    
    return increment, decrement, get_count

print("\nNonlocal keyword demonstration:")
inc, dec, get = create_counter()

print(f"Initial count: {get()}")
print(f"After increment: {inc()}")
print(f"After increment: {inc()}")
print(f"After decrement: {dec()}")
print(f"Current count: {get()}")

# Creating multiple counters
counter1_inc, counter1_dec, counter1_get = create_counter()
counter2_inc, counter2_dec, counter2_get = create_counter()

print("\nMultiple independent counters:")
print(f"Counter 1: {counter1_inc()}")
print(f"Counter 1: {counter1_inc()}")
print(f"Counter 2: {counter2_inc()}")
print(f"Counter 1: {counter1_get()}, Counter 2: {counter2_get()}")

# Nested nonlocal
def outer_func():
    x = 1
    
    def middle_func():
        nonlocal x
        x = 2
        
        def inner_func():
            nonlocal x
            x = 3
            print(f"    Inner: x = {x}")
        
        print(f"  Before inner: x = {x}")
        inner_func()
        print(f"  After inner: x = {x}")
    
    print(f"Before middle: x = {x}")
    middle_func()
    print(f"After middle: x = {x}")

print("\nNested nonlocal example:")
outer_func()

## 3. Closures and Factory Functions

Creating functions that remember their enclosing scope:

In [ ]:
# Closures and factory functions
print("Closures and Factory Functions:")
print("=" * 33)

# Basic closure
def make_multiplier(factor):
    """Create a function that multiplies by a given factor"""
    def multiplier(number):
        return number * factor  # 'factor' is captured from enclosing scope
    return multiplier

# Create different multipliers
double = make_multiplier(2)
triple = make_multiplier(3)
times_ten = make_multiplier(10)

print("Basic closure example:")
print(f"double(5) = {double(5)}")
print(f"triple(4) = {triple(4)}")
print(f"times_ten(3) = {times_ten(3)}")

# Check closure properties
print(f"\nClosure properties:")
print(f"double.__closure__: {double.__closure__}")
print(f"Captured value: {double.__closure__[0].cell_contents}")

# Closure with mutable state
def make_accumulator(initial=0):
    """Create an accumulator function"""
    total = initial
    
    def accumulate(value):
        nonlocal total
        total += value
        return total
    
    def reset():
        nonlocal total
        total = initial
    
    def get_total():
        return total
    
    # Return a tuple of functions
    return accumulate, reset, get_total

acc, reset, get_total = make_accumulator(100)

print("\nAccumulator closure:")
print(f"Initial: {get_total()}")
print(f"Add 10: {acc(10)}")
print(f"Add 5: {acc(5)}")
print(f"Add 3: {acc(3)}")
print(f"Reset and get total: {get_total()}")
reset()
print(f"After reset: {get_total()}")

# Closure with configuration
def make_validator(min_length=1, max_length=100, required_chars=None):
    """Create a customized validator function"""
    required_chars = required_chars or set()
    
    def validate(text):
        if not isinstance(text, str):
            return False, "Must be a string"
        
        if len(text) < min_length:
            return False, f"Too short (min {min_length})"
        
        if len(text) > max_length:
            return False, f"Too long (max {max_length})"
        
        if required_chars and not required_chars.issubset(set(text)):
            missing = required_chars - set(text)
            return False, f"Missing required characters: {missing}"
        
        return True, "Valid"
    
    return validate

# Create different validators
password_validator = make_validator(8, 50, {'@', '#', '$'})
username_validator = make_validator(3, 20)

test_cases = [
    ("abc", username_validator, "username"),
    ("alice123", username_validator, "username"),
    ("pass", password_validator, "password"),
    ("password123@#$", password_validator, "password"),
]

print("\nValidator closures:")
for text, validator, type_name in test_cases:
    is_valid, message = validator(text)
    status = "✓" if is_valid else "✗"
    print(f"  {status} {type_name} '{text}': {message}")

# Closure in a loop (common pitfall and solution)
print("\nClosure in loop (common pitfall):")

# Wrong way - all functions capture the same variable
functions_wrong = []
for i in range(3):
    functions_wrong.append(lambda x: x * i)  # All capture same 'i'

print("Wrong approach:")
for j, func in enumerate(functions_wrong):
    print(f"  Function {j}: 5 * i = {func(5)} (all use i={i})")

# Correct way - capture the value, not the variable
functions_correct = []
for i in range(3):
    functions_correct.append((lambda i: lambda x: x * i)(i))  # Capture value

print("\nCorrect approach:")
for j, func in enumerate(functions_correct):
    print(f"  Function {j}: 5 * {j} = {func(5)}")

# Alternative correct way using default arguments
functions_alt = []
for i in range(3):
    functions_alt.append(lambda x, multiplier=i: x * multiplier)

print("\nAlternative correct approach:")
for j, func in enumerate(functions_alt):
    print(f"  Function {j}: 5 * {j} = {func(5)}")

## 4. Namespace Management

Understanding Python's namespace system:

In [ ]:
# Namespace management
print("Namespace Management:")
print("=" * 20)

# Exploring namespaces
print("Built-in namespace (partial):")
import builtins
builtin_names = [name for name in dir(builtins) if not name.startswith('_')][:10]
print(f"  {builtin_names}...")

# Global namespace
print(f"\nGlobal namespace keys (partial):")
global_names = [name for name in globals().keys() if not name.startswith('_')][:10]
print(f"  {global_names}...")

# Local namespace
def explore_local_namespace(param1, param2="default"):
    """Explore local namespace"""
    local_var = "I'm local"
    another_local = 42
    
    print(f"  Local namespace: {list(locals().keys())}")
    print(f"  Local values: {locals()}")
    
    return local_var, another_local

print(f"\nLocal namespace exploration:")
result = explore_local_namespace("arg1", param2="arg2")

# vars() function
class SimpleClass:
    def __init__(self, name, value):
        self.name = name
        self.value = value

obj = SimpleClass("test", 123)
print(f"\nObject namespace using vars():")
print(f"  {vars(obj)}")

# Modifying namespaces (be careful!)
def namespace_manipulation():
    """Demonstrate namespace manipulation"""
    x = 1
    y = 2
    
    print(f"  Before: x={x}, y={y}")
    
    # Add variable dynamically
    locals()['z'] = 3  # This might not work as expected!
    
    # This works reliably
    exec('w = 4')
    
    print(f"  After: locals() = {locals()}")
    
    # Global namespace modification
    globals()['dynamic_global'] = "Created dynamically"

print(f"\nNamespace manipulation:")
namespace_manipulation()
print(f"Dynamic global variable: {dynamic_global}")

# Namespace isolation
def isolated_execution(code, safe_globals=None):
    """Execute code in isolated namespace"""
    if safe_globals is None:
        safe_globals = {'__builtins__': __builtins__}
    
    local_namespace = {}
    
    try:
        exec(code, safe_globals, local_namespace)
        return local_namespace, None
    except Exception as e:
        return None, str(e)

code_examples = [
    "result = 2 + 3",
    "import math; result = math.pi",
    "result = sum([1, 2, 3, 4, 5])",
    # "import os; result = os.listdir('.')"  # This would fail in restricted environment
]

print(f"\nIsolated code execution:")
for code in code_examples:
    namespace, error = isolated_execution(code)
    if error:
        print(f"  Error executing '{code}': {error}")
    else:
        print(f"  '{code}' -> result = {namespace.get('result', 'No result')}")

# Module namespaces
import math
import sys

print(f"\nModule namespaces:")
print(f"  math module attributes (sample): {dir(math)[:5]}...")
print(f"  math.pi = {math.pi}")
print(f"  sys.version_info = {sys.version_info}")

# Name resolution in classes
class NamespaceDemo:
    class_var = "I'm a class variable"
    
    def __init__(self, instance_var):
        self.instance_var = instance_var
    
    def method(self):
        method_var = "I'm a method variable"
        print(f"    Method can access:")
        print(f"      Class variable: {self.class_var}")
        print(f"      Instance variable: {self.instance_var}")
        print(f"      Method variable: {method_var}")
    
    @classmethod
    def class_method(cls):
        print(f"    Class method can access:")
        print(f"      Class variable: {cls.class_var}")
        # print(f"      Instance variable: {self.instance_var}")  # Error!
    
    @staticmethod
    def static_method():
        print(f"    Static method has limited access:")
        # print(f"      Class variable: {class_var}")  # Error!
        print(f"      Must use NamespaceDemo.class_var: {NamespaceDemo.class_var}")

print(f"\nClass namespace demonstration:")
obj = NamespaceDemo("I'm an instance variable")
obj.method()
NamespaceDemo.class_method()
NamespaceDemo.static_method()

## 5. Common Scope Pitfalls

Avoiding common mistakes with variable scope:

In [ ]:
# Common scope pitfalls
print("Common Scope Pitfalls:")
print("=" * 22)

# Pitfall 1: Late binding in loops
print("Pitfall 1: Late binding in loops")

# Problem: All functions refer to the same variable
functions = []
for i in range(3):
    functions.append(lambda: i)  # All functions capture same 'i'

print("  Problem - all functions see final value of i:")
for j, func in enumerate(functions):
    print(f"    Function {j} returns: {func()}")

# Solution 1: Use default argument to capture value
functions_fixed1 = []
for i in range(3):
    functions_fixed1.append(lambda i=i: i)

print("  Solution 1 - default argument:")
for j, func in enumerate(functions_fixed1):
    print(f"    Function {j} returns: {func()}")

# Solution 2: Use closure to capture value
functions_fixed2 = []
for i in range(3):
    functions_fixed2.append((lambda x: lambda: x)(i))

print("  Solution 2 - closure:")
for j, func in enumerate(functions_fixed2):
    print(f"    Function {j} returns: {func()}")

# Pitfall 2: Mutable default arguments
print("\nPitfall 2: Mutable default arguments")

def bad_append(item, target=[]):  # BAD: mutable default
    target.append(item)
    return target

def good_append(item, target=None):  # GOOD: use None
    if target is None:
        target = []
    target.append(item)
    return target

print("  Bad function (mutable default):")
result1 = bad_append(1)
result2 = bad_append(2)  # Modifies same list!
print(f"    First call: {result1}")
print(f"    Second call: {result2}")

print("  Good function (None default):")
result3 = good_append(1)
result4 = good_append(2)
print(f"    First call: {result3}")
print(f"    Second call: {result4}")

# Pitfall 3: Global variable modification confusion
print("\nPitfall 3: Global variable modification")

counter = 0

def confusing_function():
    print(f"    Before assignment: {counter}")  # This line causes UnboundLocalError!
    # counter = counter + 1  # This would fail!

def clear_function():
    global counter
    print(f"    Before assignment: {counter}")
    counter = counter + 1
    print(f"    After assignment: {counter}")

print("  Clear function (with global):")
clear_function()

# Pitfall 4: Closure variable modification
print("\nPitfall 4: Closure variable modification")

def create_functions():
    funcs = []
    for i in range(3):
        def func():
            # return i  # Would return 2 for all functions
            pass
        func.value = i  # Store as attribute instead
        funcs.append(func)
    return funcs

my_funcs = create_functions()
print("  Function attributes to avoid closure issues:")
for j, func in enumerate(my_funcs):
    print(f"    Function {j} has value: {func.value}")

# Pitfall 5: Class variable vs instance variable confusion
print("\nPitfall 5: Class vs instance variables")

class Counter:
    count = 0  # Class variable
    
    def __init__(self):
        Counter.count += 1  # Modify class variable
        self.instance_count = 0  # Instance variable
    
    def increment_class(self):
        Counter.count += 1
    
    def increment_instance(self):
        self.instance_count += 1

c1 = Counter()
c2 = Counter()

print(f"  After creating 2 instances:")
print(f"    Class count: {Counter.count}")
print(f"    c1 instance count: {c1.instance_count}")
print(f"    c2 instance count: {c2.instance_count}")

c1.increment_class()
c1.increment_instance()

print(f"  After c1 increments:")
print(f"    Class count: {Counter.count}")
print(f"    c1 instance count: {c1.instance_count}")
print(f"    c2 instance count: {c2.instance_count}")

# Debugging scope issues
print("\nDebugging tools:")

def debug_scopes():
    local_var = "local"
    
    def inner():
        inner_var = "inner"
        print(f"    Inner locals: {list(locals().keys())}")
        print(f"    Inner globals (sample): {list(globals().keys())[:5]}...")
    
    print(f"  Outer locals: {list(locals().keys())}")
    inner()

debug_scopes()

## Summary

In this notebook, you learned about:

✅ **LEGB Rule**: Local, Enclosing, Global, Built-in scope resolution  
✅ **Global/Nonlocal**: Keywords for modifying outer scope variables  
✅ **Closures**: Functions that capture their enclosing environment  
✅ **Namespaces**: Python's system for organizing names  
✅ **Common Pitfalls**: Late binding, mutable defaults, scope confusion  
✅ **Best Practices**: Writing clean, maintainable scoped code  

### Key Takeaways:
1. Python follows LEGB rule for variable resolution
2. Use `global` and `nonlocal` sparingly and explicitly
3. Closures capture variables by reference, not value
4. Avoid mutable default arguments
5. Be explicit about scope intentions in your code
6. Use debugging tools like `locals()` and `globals()` when needed

### Next Topic: 16_error_handling.ipynb
Learn about exceptions, try-catch blocks, and robust error handling.